In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [5]:
# Function to extract Product Title
def get_title(soup):

    try:
        # Outer Tag Object
        title = soup.find("span", attrs={"id":'productTitle'})
        
        # Inner NavigatableString Object
        title_value = title.text

        # Title as a string value
        title_string = title_value.strip()

    except AttributeError:
        title_string = ""

    return title_string

# Function to extract Product Price
def get_price(soup):

    try:
        price = soup.find("span", attrs={'class':'a-offscreen'}).string.strip()
    

    except AttributeError:

        try:
            # If there is some deal price
            price = soup.find("span", attrs={'aria-hidden':'true'}).string.strip()

        except:
            price = ""

    return price

# Function to extract Product Rating
def get_rating(soup):

    try:
        rating = soup.find("i", attrs={'class':'a-icon a-icon-star a-star-4-5'}).string.strip()
    
    except AttributeError:
        try:
            rating = soup.find("span", attrs={'class':'a-icon-alt'}).string.strip()
        except:
            rating = ""

    return rating

# Function to extract Number of User Reviews
def get_review_count(soup):
    try:
        review_count = soup.find("span", attrs={'id':'acrCustomerReviewText'}).string.strip()

    except AttributeError:
        review_count = ""

    return review_count

# Function to extract Availability Status
def get_availability(soup):
    try:
        available = soup.find("div", attrs={'id':'availability'})
        available = available.find("span").string.strip()

    except AttributeError:
        available = "Not Available"	

    return available

In [6]:
if __name__ == '__main__':

    # add your user agent 
    HEADERS = ({'User-Agent':'', 'Accept-Language': 'en-US, en;q=0.5'})

    # The webpage URL
    URL = "https://www.amazon.com/s?k=sunscreen&ref=nb_sb_noss"

    # HTTP Request
    webpage = requests.get(URL, headers=HEADERS)

    # Soup Object containing all data
    soup = BeautifulSoup(webpage.content, "html.parser")

    # Fetch links as List of Tag Objects
    links = soup.find_all("a", attrs={'class':'a-link-normal s-no-outline'})

    # Store the links
    links_list = []

    # Loop for extracting links from Tag Objects
    for link in links:
            links_list.append(link.get('href'))

    d = {"title":[], "price":[], "rating":[], "reviews":[],"availability":[]}
    
    # Loop for extracting product details from each link 
    for link in links_list:
        new_webpage = requests.get("https://www.amazon.com" + link, headers=HEADERS)

        new_soup = BeautifulSoup(new_webpage.content, "html.parser")
        
        # Function calls to display all necessary product information
        d['title'].append(get_title(new_soup))
        d['price'].append(get_price(new_soup))
        d['rating'].append(get_rating(new_soup))
        d['reviews'].append(get_review_count(new_soup))
        d['availability'].append(get_availability(new_soup))

    
    amazon_df = pd.DataFrame.from_dict(d)
    amazon_df['title'].replace('', np.nan, inplace=True)
    amazon_df = amazon_df.dropna(subset=['title'])
    amazon_df.to_csv("amazon_data.csv", header=True, index=False)

In [7]:
amazon_df

,title,price,rating,reviews,availability
0,CeraVe 100% Mineral Sunscreen SPF 50 | Face Su...,$13.63,3.9 out of 5 stars,"14,351 ratings",In Stock
1,Neutrogena Ultra Sheer Dry-Touch Water Resista...,$8.99,4.6 out of 5 stars,"13,819 ratings",In Stock
2,CeraVe 100% Mineral Sunscreen SPF 30 | Face Su...,$14.97,4.1 out of 5 stars,"5,705 ratings",In Stock
3,"Banana Boat Ultra Sport Sunscreen Lotion, Broa...",$10.97,4.7 out of 5 stars,"13,052 ratings",In Stock
4,"Coppertone SPORT Sunscreen Spray SPF 50, Water...",Page 1 of 1,4.8 out of 5 stars,"8,267 ratings",
...,...,...,...,...,...
69,Aveeno Protect + Hydrate Moisturizing Body Sun...,$26.11,4.6 out of 5 stars,"2,259 ratings",In Stock
70,Supergoop! PLAY Everyday Lotion SPF 50-18 fl o...,Page 1 of 1,4.7 out of 5 stars,"6,356 ratings",Currently unavailable.
71,"Coppertone SPORT Sunscreen SPF 30 Lotion, Wate...",$7.97,4.8 out of 5 stars,"2,155 ratings",In Stock
72,Banana Boat UltraMist Kids MAX Protect & Play ...,Page 1 of 1,4.8 out of 5 stars,"2,799 ratings",
